In [3]:
import os
import tgt
from parselmouth.praat import call

class PraatTextgrid(object):
    def __init__(self, name):
        self.name = name

    @staticmethod
    def get_tiernames_from_tgfile(read_file):
        tg = tgt.read_textgrid(read_file, encoding='utf-16')
        tier_names = [tier.name for tier in tg.tiers]
        return tg, tier_names
        
    @staticmethod
    def read_tier_from_tg(tg, tier_name):
        tag_tier = tg.get_tier_by_name(tier_name)
        return tag_tier

    @staticmethod
    def write_tgfile(outfile, tiers):
        tg = tgt.core.TextGrid()
        for tier in tiers:
            tg.add_tier(tier)
        tgt.io.write_to_file(tg, outfile, format = 'long', encoding='utf-8')

    @staticmethod
    def create_interval_tier(start_time, end_time, name, intervals):
        tier = tgt.core.IntervalTier(start_time, end_time, name)
        for interval in intervals:
            # interval = [start_time, end_time, text]
            annotation = tgt.core.Interval(interval[0], interval[1], interval[2])
            tier.add_interval(annotation)
        return tier

    @staticmethod
    def create_point_tier(start_time, end_time, name, points):
        tier = tgt.core.IntervalTier(start_time, end_time, name)
        for interval in intervals:
            # interval = [time, text]
            annotation = tgt.core.Interval(interval[0], interval[1])
            tier.add_interval(annotation)
        return tier
        
    @staticmethod
    def create_silence_tg(wav_path, output_sil_tg_path):
        wav_praat_read = call('Read from file', wav_path)
        sil_tier = call(wav_praat_read, 'To TextGrid (silences)', 100, 0, -25, 0.1, 0.1, "SIL", "")
        output = call(sil_tier, 'Write to text file', output_sil_tg_path)
    
    @staticmethod
    def get_wave_duration(wav_file):
        import sox
        duration = sox.file_info.duration(wav_file)
        return duration

In [98]:
dir_path = '/home/jovyan/wm-insur-call-qa/insur_data_202012_result/20201201'
file_list = os.listdir(dir_path)



for file_name in file_list[1:]:
    test_read_tg = os.path.join(f'{dir_path}/{file_name}')
    textgrid, tier_names = PraatTextgrid.get_tiernames_from_tgfile(test_read_tg)

    if not ('Esun' or 'customer') in tier_names:   # [ ] 確認
        continue

    esun_annotation = PraatTextgrid.read_tier_from_tg(textgrid, 'Esun').annotations
    cust_annotation = PraatTextgrid.read_tier_from_tg(textgrid, 'customer').annotations
    
    init, last_signal = 1, -1
    esun_sentences, cust_sentences = '', ''
    while esun_annotation or cust_annotation:
        if init:
            esun_sentence = esun_annotation.pop(0)
            cust_sentence = cust_annotation.pop(0)

        # print('###', esun_annotation, cust_annotation)
        this_signal = 0 if esun_sentence.start_time > cust_sentence.start_time else 1
        print(this_signal, last_signal)   
        if this_signal == 0 and last_signal == 1:
            print('#RST', esun_sentences, '/', cust_sentences)
            esun_sentences, cust_sentences = '', ''
        elif last_signal == 0:    
            esun_sentences += esun_sentence.text
            esun_sentence = esun_annotation.pop(0)
        elif last_signal == 1:
            cust_sentences += cust_sentence.text
            cust_sentence = cust_annotation.pop(0)
        
        last_signal = this_signal
        init = None 
    break

1 -1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1


IndexError: pop from empty list

In [22]:
import pandas as pd
def show_result(df):
    tmp_df = df.loc[df['sd_type']=='no_vad', 'Esun_correct_rate']
    print('[no_vad / esun    ] mean:', tmp_df.mean(),'std:', tmp_df.std())
    tmp_df = df.loc[df['sd_type']=='vad', 'Esun_correct_rate']
    print('[   vad / esun    ] mean:', tmp_df.mean(),'std:', tmp_df.std())
    tmp_df = df.loc[df['sd_type']=='no_vad', 'customer_correct_rate']
    print('[no_vad / customer] mean:', tmp_df.mean(),'std:', tmp_df.std())
    tmp_df = df.loc[df['sd_type']=='vad', 'customer_correct_rate']
    print('[   vad / customer] mean:', tmp_df.mean(),'std:', tmp_df.std())

In [23]:
df = pd.read_csv('/home/jovyan/wm-insur-call-qa/owen/data/eval_total_model_1s.tsv', sep='\t')
show_result(df)

[no_vad / esun    ] mean: 0.8298963464907404 std: 0.12329809665218401
[   vad / esun    ] mean: 0.8285688350130987 std: 0.09464108524220206
[no_vad / customer] mean: 0.3937373569934391 std: 0.28183126833313177
[   vad / customer] mean: 0.23202619141583336 std: 0.2229519530109746


In [26]:
df = pd.read_csv('/home/jovyan/wm-insur-call-qa/owen/data/eval_total_model_1s_with_400p_and_hr.tsv', sep='\t')
show_result(df)


[no_vad / esun    ] mean: 0.8516633511905198 std: 0.15049344485321198
[   vad / esun    ] mean: 0.7573187828600317 std: 0.1390150815065076
[no_vad / customer] mean: 0.40393234780261933 std: 0.33978014850617216
[   vad / customer] mean: 0.2906984633769666 std: 0.20514428840189006


In [27]:
df = pd.read_csv('/home/jovyan/wm-insur-call-qa/owen/data/eval_total_model_1s_margin1_ep100_ppl8.tsv', sep='\t')
show_result(df)

[no_vad / esun    ] mean: 0.8945761694952284 std: 0.08565990084317135
[   vad / esun    ] mean: 0.7397765170649842 std: 0.11549236429488308
[no_vad / customer] mean: 0.46773202078346593 std: 0.30970211600506375
[   vad / customer] mean: 0.3286975297803478 std: 0.24300114799176095


In [28]:
df = pd.read_csv('/home/jovyan/wm-insur-call-qa/owen/data/eval_total_model_1s_margin1_ep100_fine_tune_8.tsv', sep='\t')
show_result(df)

[no_vad / esun    ] mean: 0.8658877240172569 std: 0.10512343105063517
[   vad / esun    ] mean: 0.8109129074258384 std: 0.10202844495383796
[no_vad / customer] mean: 0.504540506893644 std: 0.2867028350502723
[   vad / customer] mean: 0.26126618261050294 std: 0.2627614289366204


In [29]:
df = pd.read_csv('/home/jovyan/wm-insur-call-qa/owen/data/eval_total_model_1s_margin1_ep100_fine_tune_733.tsv', sep='\t')
show_result(df)

[no_vad / esun    ] mean: 0.8821807629925459 std: 0.13010822036431746
[   vad / esun    ] mean: 0.776316854624083 std: 0.11931582475916283
[no_vad / customer] mean: 0.4273188635239333 std: 0.34767367432027835
[   vad / customer] mean: 0.2818870022473118 std: 0.20915927038026147


In [15]:
a = [1, 3, 4, 6, 7]
# for x in range(0, len(a), 2):
#     print(x)

2

### 音檔去重


In [4]:
import sys

sys.path.append('/home/jovyan/wm-insur-call-qa/owen/evaluation')
from textgrid_open import get_textgrid_dict

In [5]:
conversation = get_textgrid_dict('/home/jovyan/wm-insur-call-qa/insur_data_202012_result/20201203/220R0000000127160697593000042.TextGrid')
conversation

{'Esun': {'times': [[12.717797714514836, 17.77037389735365],
   [17.77037389735365, 20.0403718925421],
   [21.94424117882919, 22.70090717722534],
   [24.89767943063352, 25.31262530072173],
   [26.248772168433277, 28.909307453116515],
   [29.910059257446907, 30.655749999999998],
   [30.655749999999998, 36.64682750123199],
   [36.64682750123199, 41.455317878136555],
   [46.856875, 51.65790420650623],
   [51.65790420650623, 55.68532000442122],
   [55.68532000442122, 59.248972771061155],
   [59.248972771061155, 64.275],
   [64.275, 66.2784435685288],
   [69.178375, 73.30811477943497],
   [75.94424148352479, 81.70897690615362],
   [81.70897690615362, 84.83327522211191],
   [84.83327522211191, 86.63048866150059],
   [90.04768994457997, 90.53586155644845],
   [93.78220277537388, 99.37176773126801],
   [99.37176773126801, 101.64176572645647],
   [102.52047462781974, 104.18025810817258]],
  'texts': ['呃您好,這裡是玉山銀行總行保險代理部,敝姓張,員工編號二一一三七',
   '請問是余以萱小姐本人嗎',
   '您好,要跟您',
   '呃',
   '呃不會,大概一分鐘,請問可以嗎'

In [6]:
esun_time = conversation['Esun']['times']
cust_time = conversation['customer']['times']
esun_time, cust_time

([[12.717797714514836, 17.77037389735365],
  [17.77037389735365, 20.0403718925421],
  [21.94424117882919, 22.70090717722534],
  [24.89767943063352, 25.31262530072173],
  [26.248772168433277, 28.909307453116515],
  [29.910059257446907, 30.655749999999998],
  [30.655749999999998, 36.64682750123199],
  [36.64682750123199, 41.455317878136555],
  [46.856875, 51.65790420650623],
  [51.65790420650623, 55.68532000442122],
  [55.68532000442122, 59.248972771061155],
  [59.248972771061155, 64.275],
  [64.275, 66.2784435685288],
  [69.178375, 73.30811477943497],
  [75.94424148352479, 81.70897690615362],
  [81.70897690615362, 84.83327522211191],
  [84.83327522211191, 86.63048866150059],
  [90.04768994457997, 90.53586155644845],
  [93.78220277537388, 99.37176773126801],
  [99.37176773126801, 101.64176572645647],
  [102.52047462781974, 104.18025810817258]],
 [[11.863497393744987, 12.717797714514836],
  [20.821446471531676, 26.045875],
  [28.909307453116515, 30.655749999999998],
  [42.041123812378736,

In [36]:
import numpy as np
import math
def determine_abs(x, y):
    x, y = [i+1 if i == 0 else i for i in [x, y]]
    return np.sign(x * y)

In [38]:

x_time, y_time = esun_time, cust_time

def get_non_overlapping_time(x_time, y_time):
    only_x_time = []
    for x_start, x_end in x_time:
        revised_tag = False
        for y_start, y_end in y_time:
            if x_end <= y_start or y_end <= x_start:
                continue
            revised_tag = True
            y_start_relative = determine_abs(*[y_start - e for e in [x_start, x_end]])
            y_end_relative = determine_abs(*[y_end - e for e in [x_start, x_end]])
            if (y_start_relative, y_end_relative) == (-1, -1):
                only_x_time.append((x_start, y_start))
                only_x_time.append((y_end, x_end))
            elif (y_start_relative, y_end_relative) == (-1, 1):
                only_x_time.append((x_start, y_start))
            elif (y_start_relative, y_end_relative) == (1, -1):
                only_x_time.append((y_end, x_end))
            else:
                # 完全被覆蓋
                pass
        if not revised_tag:
            only_x_time.append((x_start, x_end))
    return only_x_time

get_non_overlapping_time

[(12.717797714514836, 17.77037389735365),
 (17.77037389735365, 20.0403718925421),
 (26.248772168433277, 28.909307453116515),
 (30.655749999999998, 36.64682750123199),
 (36.64682750123199, 41.455317878136555),
 (46.856875, 51.65790420650623),
 (51.65790420650623, 55.68532000442122),
 (55.68532000442122, 59.248972771061155),
 (59.248972771061155, 62.714991215327395),
 (69.178375, 73.30811477943497),
 (75.94424148352479, 81.70897690615362),
 (81.70897690615362, 84.83327522211191),
 (84.83327522211191, 86.2399513720058),
 (93.78220277537388, 97.41908128379407),
 (99.37176773126801, 101.64176572645647),
 (102.95982907850137, 104.18025810817258)]